In [1]:
import pandas as pd
import numpy as np

In [2]:
# This is my custom table, for each team-game, derived from the original gamelog table
glt = pd.read_parquet('../data/mine/gl_teams.parquet')

In [3]:
# Narrow to games where all the scoring is from solo HRs (and regular-season games only)
all_solos = glt[(glt['hr']==glt['runs_scored'])&(glt['game_type']=='RS')]

In [4]:
# identify consecutive games by subtracting the cumcount from team_game_number (two increasing counts)
all_solos['streak_id'] = all_solos['team_game_number']-all_solos.groupby(['team', 'yr']).cumcount()

/Users/vkumar/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
# Now find streaks, finding the total HR, the number of games, and the date of the streak's first game
all_solos.groupby(['team', 'yr', 'streak_id']) \
    .agg({'hr': sum, 'streak_id': np.size, 'date': min}) \
    .rename(columns={'streak_id': 'num_games', 'date': 'start_dt'}) \
    .sort_values(by=['num_games', 'hr'], ascending=False)

hr  num_games   start_dt
team yr   streak_id                           
CHN  2018 102        5.0          5 2018-08-16
CIN  2018 127        5.0          5 2018-09-12
NYN  1963 1          3.0          5 1963-04-09
KCA  2017 110        2.0          5 2017-08-24
SLN  1904 142        2.0          5 1904-10-06
...                  ...        ...        ...
WS2  1971 76         0.0          1 1971-07-22
          109        0.0          1 1971-08-29
          115        0.0          1 1971-09-06
          127        0.0          1 1971-09-24
WS3  1872 1          0.0          1 1872-04-18

[34986 rows x 3 columns]